### Google ebook reader app - Part C

- In this last part of the Example project assignment we wil:
1) Instead of the books.csv data, we will now use the Google Book API (no API key needed!) to search for free ebooks by author or by title
2) Refactor the TkInter GUI from part B to use the Google API search 
3) Add a button to open the result of the search withing a browser-based ebook reader. (Other GUI enhancements, such as showing a thumbnail for the result are optional.)

<p>

##### Google Books API: https://developers.google.com/books/docs/v1/using


- https://developers.google.com/books/docs/v1/using#WorkingVolumes shows how to use a HTTP GET request to search for books in various ways and with various filters.
- https://developers.google.com/books/docs/v1/using#api_params is a summary of all parameters
- You can dig into these later, for now, let's look at a simple, functional example of a query and analyze its response

<p>

- We will simulate the query using the requests library (https://requests.readthedocs.io/en/master/), which is a very popular library for making HTTP requests. It is not part of the standard library, so you will need to install it with pip install requests below.
- If the query was successful, the response will be a JSON object, which we can parse with the json library (https://docs.python.org/3/library/json.html), which is part of the standard library.
- The JSON object will contain a list of books (items), each of which is a dictionary. We can use the pprint library (https://docs.python.org/3/library/pprint.html) to print the dictionary in a more readable format.
- Note: we are going to use this API w/o having registered for an API key. This usually works but it is possible that you may get a "resource exhausted" error, meaning worldwide more that 20,000,000 API-key-less requests have been made in the last 24 hours. If that happens, you will need to wait a day and try again (although I don't know in what timezone Google counts the 24 hours ...).

In [ ]:
%pip install requests --upgrade

In [1]:
import requests
from pprint import pprint

query = 'intitle:"Human Computer Interaction"' # search for books by title, 
# multi word search terms have to wrapped into "", so use '' to make a string with "" inside
#query = "subject:Biology" # search for books by subject
#query = 'inauthor:"Agatha Christie"'
query= 'inauthor:"Amy Krouse Rosenthal" +intitle:"I Wish You More"'  
url = "https://www.googleapis.com/books/v1/volumes"
params = {  "q": query, 
            "maxResults": 3, # max for this is 40
            #"filter":"free-ebooks", # comment this out to get all results (not just free ebooks)
            "orderBy":"relevance", 
            #"orderBy":"newest",
            "langRestrict":"en"
            }
response = requests.get('https://www.googleapis.com/books/v1/volumes', params=params)
raw_response = requests.get(url, params=params) # make request to API
display(raw_response) # 200 means success
print(raw_response.url) # click on this to see the data in a online json viewer


<Response [200]>

https://www.googleapis.com/books/v1/volumes?q=inauthor%3A%22Amy+Krouse+Rosenthal%22+%2Bintitle%3A%22I+Wish+You+More%22&maxResults=3&orderBy=relevance&langRestrict=en


In [2]:
# write JSON payload into a .json file so we can look it in a separate viewer
with open("book.json", "w+") as fo:
    print(raw_response.text, file=fo)

In [ ]:
items_dict = raw_response.json() # .json() converts json payload to python dictionary
print(len(items_dict["items"])) # how many items?
pprint(items_dict["items"][0], width=100) # show item 1

- in our dictionary, the key 'items' contains a list of books. (totalItems is the total number of books that the google book store has for this query)
- let's loop over all items
- Within each item we can see that the title is in the __volumeInfo__ dictionary, and the authors are in a list in the authors key of that dictionary. The date and description are also in the volumeInfo dictionary. (Note that not all books have a description!)

```json
"volumeInfo": {
        "title": "Interaction Techniques and Technologies in Human-Computer Interaction",
        "authors": [
          "Constantine Stephanidis",
          "Gavriel Salvendy"
        ],
        "publishedDate": "2024-08-07",
        "description": "The reader of this book will gain an in-depth understanding ..."
}
```

- (volumeInfo also has a link to a thumbnail image for the book, which we could use to display a thumbnail in our GUI.)

In [ ]:
if items_dict['totalItems'] > 0: # will be 0 if nothing was found
    for book in items_dict["items"]:
        #pprint(book)
        #print()
        volumeInfo = book["volumeInfo"]
        author = volumeInfo["authors"][0] # get the first author only
        title = volumeInfo["title"]
        description = volumeInfo["description"]
        print(f"{title}\nby {author}\n{description}\n\n")

- Note: it's possible that an item doesn't have a description, so we will need to defend against that. (We could also check for other keys that might be missing, such as authors.)   


#### Finding the ISBN number
- if we want to open the book in a browser-based ebook reader, we will need to find the ISBN number for the book. 
- The ISBN number is a unique identifier for the book, and is used in the URL for the ebook reader.
- ISBN10 can also be used to get a link to the book on amazon: http://www.amazon.com/dp/1032370033 
- The ISBN number is in the `industryIdentifiers` list, which contains a dictionary for each type of identifier. 
- `industryIdentifiers` itself is a key in the dict `book["volumeInfo"]`

In [ ]:
# using the last book from the loop above as example
pprint(book["volumeInfo"]['industryIdentifiers'])


- Here, I'm using try/except to handle the case where there is no industryIdentifiers
- if you do have a industryIdentifiers, it's still possible that there is no ISBN 10 or 13 in it (there are other types of identifiers, but we can't use those for the ebook reader)
- the use try/except for cases where there's no industryIdentifiers, or no ISBN 10 or 13 
- This is also how we could defend against missing authors or description. 
- Finally, it is possible to get duplicates based on title and author (maybe 2 different editions?)
- we will prevent this by adding a title and author tuple `(title, author)` to a set (which cannot contain duplicates) and compare a new book against this set. If we have it already, we skip it otherwise we add it.

In [ ]:
seen_books = set()  # create a set to store the books we've seen

if items_dict['totalItems'] > 0:
    for book in items_dict["items"]:
        volumeInfo = book["volumeInfo"]
        author = volumeInfo["authors"][0] # first author only
        title = volumeInfo["title"]
        
        # prevent duplicates
        book_info = (title, author)  # create a tuple with  title and author
        if book_info in seen_books:  # if we've already seen this book, skip it
            continue
        seen_books.add(book_info)  # otherwise add the book to the set 

        print(f"{title} by {author}")
        try:
            identifier_list = volumeInfo["industryIdentifiers"]
        except KeyError:
            print(": no industryIdentifiers for this book")
            continue # skip this book if no chance for an ISBN
        else:
            isbn = None
            for identifier in identifier_list:
                if identifier["type"] == "ISBN_13":
                    isbn = identifier["identifier"]
                    print(f"ISBN13={isbn}")
                elif identifier["type"] == "ISBN_10": # prefer ISBN10
                    isbn = identifier["identifier"]
                    print(f"ISBN10={isbn}")
            if isbn is None:
                print("no ISBN found")
        print()

### Digging deeper
- change the code below so that it also prints the values for publicDomain and textToSpeechPermission
- if you use the json viewer, be aware that it will show True as true and False as false ...  

In [ ]:
if items_dict['totalItems'] > 0:
    for book in items_dict["items"]:
        volumeInfo = book["volumeInfo"]
        author = volumeInfo["authors"][0] # first author only
        title = volumeInfo["title"]

        # your code here
        inpublicDomain = ???  # will be True or False
        textToSpeechPermission = ??? # will be True or False

        print(f"{title} by {author}, Is in public domain? {inpublicDomain}, text-to_speech is {textToSpeechPermission}")

#### My solution (unhide this cell by clicking to the left of My)

In [ ]:
if items_dict['totalItems'] > 0:
    for book in items_dict["items"]:
        volumeInfo = book["volumeInfo"]
        author = volumeInfo["authors"][0] # first author only
        title = volumeInfo["title"]

        # your code here
        accessInfo = book["accessInfo"]
        inpublicDomain = accessInfo["publicDomain"]  
        textToSpeechPermission = accessInfo["textToSpeechPermission"]


        print(f"{title} by {author}, Is in public domain? {inpublicDomain}, text-to_speech is {textToSpeechPermission}")

### Wrapping the book search code in a function
- let's wrap the search code in a function, so we can test it with different queries
- Our app will only display one result, but, given that we could have missing data, let's request 10 results (sorted by relevance) and loop over them until we find one that has all the data we need. 
- Note that I added another defensive measure by converting potentially unprintable unicode characters to ascii. (This is not strictly necessary, but it makes the output a little cleaner.)
- For now we'll print out the title, author, description, and ISBN number for this result but we also need to think about how the book results data connects to the GUI.
1) we could return title, author, description, and ISBN number either in a list or in a dictionary
2) we could wait until this function has been integrated into the GUI class as a method and then store each information as a class attribute

- I prefer the 2. option, so let's just print out the information for now (and not store it) and test if the function works for various queries

In [ ]:
# function to get info on a book from Google Books API
import requests
def get_books(query):

    url = "https://www.googleapis.com/books/v1/volumes"

    # create request url from parameters
    params = {"q": query, "maxResults": 10, 
              #"filter":"free-ebooks", 
              "orderBy":"relevance", 
              #"orderBy":"newest",
              "langRestrict":"en"}
    raw_response = requests.get(url, params=params) # make request (will be in json format)
    
    # bail out if we get an error
    if raw_response.status_code != 200:
        print(f"Error: {raw_response.status_code}")
        return None
    items = raw_response.json() # convert json to dictionary
    if items['totalItems'] == 0:
        return("Nothing found!")
    
    for book in items["items"]:
        volumeInfo = book["volumeInfo"] # dictionary of book info
        title = volumeInfo["title"]
        try:
            author = volumeInfo["authors"][0] # list of authors, only want first
        except KeyError: # not all books have an author
            author = "No author"
        try: # not all books have a description
            description = volumeInfo["description"]
        except KeyError:    
            description = "No description"

        try:
            identifier_list = volumeInfo["industryIdentifiers"]
        except KeyError:
            continue # skip this book if no chance for an ISBN
        else:
            isbn = None
            for identifier in identifier_list:
                if identifier["type"] == "ISBN_13":
                    isbn = identifier["identifier"]
                elif identifier["type"] == "ISBN_10":
                    isbn = identifier["identifier"]
            if isbn is None: 
                continue

        # convert all words to latin to avoid unicode errors
        description = description.encode("latin-1", "ignore").decode("latin-1")
        title = title.encode("latin-1", "ignore").decode("latin-1")
        author = author.encode("latin-1", "ignore").decode("latin-1")
        
        # return a string with the book info for the first good book we find
        # this also breaks out of the loop
        return f"{title}\n{author}\nISBN:{isbn}\n{description}\n\n"     

In [ ]:
query_list = ['inauthor:"Agatha Christie"', 
              'intitle:"Linux System Administration"',  
              "intitle:Gobbledigook", 
              "subject:Literature",  # another way to search, which we won't use (but you could implement!)
              "Seven Devils"] # without a prefix, searches all fields
for query in query_list:
    print(query)
    print(get_books(query))

### Opening the ebook in a browser-based ebook reader
- google_ebook_reader.html is a simple HTML file that uses the Google Books API to open an ebook in a browser-based ebook reader
- The ISBN number is in the URL for the ebook reader, so we can use the webbrowser library (https://docs.python.org/3/library/webbrowser.html) to open the ebook in a browser-based ebook reader

<p>

- To programatically create a similar html file with a specific isbn (which we have as result of a successful search) we need to insert (inline) the ISBN number into the HTML file before we have python open it in the browser.
- Jinja2 (https://jinja.palletsprojects.com/en/2.11.x/) is a popular templating library that can do this, but it is not part of the standard library, so you will need to install it with pip install jinja2 below.

In [ ]:
%pip install jinja2 --upgrade

- The function below uses Jinja2 to create a new html file with the ISBN number (as a string) inserted into the template. 
- the critical lines in the template are:
``` html
<script>
    var viewer = new google.books.DefaultViewer(document.getElementById('viewerCanvas'));
    viewer.load('ISBN:{{ isbn }}');
</script>
```
- the `{{ isbn }}` is replaced by the isbn number string when the template is rendered. This done via the context dictionary, which contains the isbn number as a key-value pair.
- with 9780425067949 the html file looks like this:
``` html    
<script>
    var viewer = new google.books.DefaultViewer(document.getElementById('viewerCanvas'));
    viewer.load('ISBN:9780425067949');
</script>
```     

In [1]:
import webbrowser
from jinja2 import Environment, FileSystemLoader
import os.path

def open_ebook(isbn, title, author):
    # Create a Jinja2 environment with a template folder and load the template
    env = Environment(loader=FileSystemLoader('.')) # in current folder
    template = env.get_template('google_ebook_reader_template.html')

    # Create a context dictionary with the ISBN value
    context = {'isbn': isbn}

    # Render the template with the context
    rendered_template = template.render(context)

    # Create a temporary HTML file (you can use a unique name if needed)
    # this needs to be a absolute file path
    temp_html_file = os.path.abspath(f'{title}_by_{author}_ebook.html')

    # Write the rendered content to the temporary HTML file
    with open(temp_html_file, 'w') as file:
        file.write(rendered_template)

    # Open the temporary HTML file in a web browser
    # file:// is the equivalent to https:// for local html files
    webbrowser.open('file://' + temp_html_file) 


# test it with one of our earlier results: The Murder on the Links by Agatha Christie ISBN:9780425067949
open_ebook("9780425067949", "The Murder on the Links", "Agatha Christie")